In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Vertex AI 迁移：AutoML 视频对象跟踪

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-object-tracking-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-object-tracking-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-object-tracking-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练AutoML视频物体跟踪模型并进行批量预测。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[视频数据的物体跟踪](https://cloud.google.com/vertex-ai/docs/training-overview#object_tracking_for_videos)。

### 目标

在本教程中，您将学习如何使用`AutoML`训练视频模型，并使用`Vertex AI Batch Prediction`进行批量预测。

本教程使用以下谷歌云ML服务：

- `AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 训练一个AutoML视频目标跟踪模型。
- 进行批量预测。

数据集

本教程使用的数据集是Traffic。您在本教程中使用的数据集版本存储在一个公共云存储桶中。

### 成本

本教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用量生成成本估算。

安装所需的包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform  \
                                 google-cloud-storage \
                                 tensorflow

只有协作：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目 ID

**如果您不知道您的项目 ID**，请尝试以下方法：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您处于实时教程会话中，您可能正在使用共享测试账户或项目。为避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 认证您的 Google 云账户

根据您的 Jupyter 环境，您可能需要手动认证。请按照以下相关说明进行操作。

**1. Vertex AI 工作台**
* 无需操作，因为您已经认证过了。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples，了解如何向您的服务账户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常数

In [ ]:
import os

import google.cloud.aiplatform as aip

初如化Python的Vertex AI SDK

为您的项目和对应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"

快速浏览您的数据

本教程使用存储在公共云存储桶中的Traffic数据集的一个版本，使用CSV索引文件。

首先快速浏览数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

创建一个数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)
### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法为`Dataset`资源创建数据集，需要以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：要导入数据项到`Dataset`资源中的一个或多个数据集索引文件列表。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.VideoDataset.create(
    display_name="Traffic" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.video.object_tracking,
)
if os.getenv("IS_TESTING"):
    import time

    time.sleep(30)

print(dataset.resource_name)

INFO：google.cloud.aiplatform.datasets.dataset：创建VideoDataset
INFO：google.cloud.aiplatform.datasets.dataset：创建VideoDataset支持的LRO：projects/759209241365/locations/us-central1/datasets/5948525032035581952/operations/6913187331100901376
INFO：google.cloud.aiplatform.datasets.dataset：VideoDataset已创建。资源名称：projects/759209241365/locations/us-central1/datasets/5948525032035581952
INFO：google.cloud.aiplatform.datasets.dataset：在另一个会话中使用此VideoDataset：
INFO：google.cloud.aiplatform.datasets.dataset：ds = aiplatform.VideoDataset('projects/759209241365/locations/us-central1/datasets/5948525032035581952')
INFO：google.cloud.aiplatform.datasets.dataset：导入VideoDataset数据：projects/759209241365/locations/us-central1/datasets/5948525032035581952
INFO：google.cloud.aiplatform.datasets.dataset：导入VideoDataset数据支持的LRO：projects/759209241365/locations/us-central1/datasets/5948525032035581952/operations/6800597340416638976

训练一个模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### [自动机器学习 API 训练](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLVideoTrainingJob`类创建一个AutoML训练管道，需要设定以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：为模型训练指定的任务类型。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标跟踪模型。
  - `action_recognition`：视频动作识别模型。

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
dag = aip.AutoMLVideoTrainingJob(
    display_name="traffic_" + UUID,
    prediction_type="object_tracking",
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLVideoTrainingJob object at 0x7fc3b6c90f10> 

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLVideoTrainingJob 对象在 0x7fc3b6c90f10处>

#### 运行训练管道

接下来，您可以通过调用`run`方法运行DAG以启动训练作业，参数如下：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：训练模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。

完成`run`方法后，会返回`Model`资源。

训练管道的执行时间最长可达20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="traffic_" + UUID,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

示例输出：

    信息：google.cloud.aiplatform.training_jobs：查看训练：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/6090621516762841088?project=759209241365
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    ...
    信息：google.cloud.aiplatform.training_jobs：AutoMLVideoTrainingJob 运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088
    信息：google.cloud.aiplatform.training_jobs：模型可在 projects/759209241365/locations/us-central1/models/1899701006099283968 找到。

评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [项目.位置.模型.评估.列表](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 评估模型评分
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=traffic_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

示例输出：

    名称：“projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824”
    指标模式 URI：“gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml”
    指标 {
      结构值 {
        字段 {
          键： “auPrc”
          值 {
            数值：0.9891107
          }
        }
        字段 {
          键： “confidenceMetrics”
          值 {
            列表值 {
              对象 {
                字段 {
                  键： “precision”
                  值 {
                    数值：0.2
                  }
                }
                字段 {
                  键： “recall”
                  值 {
                    数值：1.0
                  }
                }
              }
            }

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### 获取测试项

现在对您的Vertex模型进行批量预测。您可以在数据集中随意选择一些示例作为测试项。不必担心这些示例可能被用来训练模型 - 我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

cols_1 = test_items[0].split(",")
cols_2 = test_items[1].split(",")
if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[2])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[1])
    test_label_2 = str(cols_2[1])


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 创建一个批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中，您使用JSONL格式。对于JSONL文件，您为每个视频的每一行创建一个字典条目。该字典包含以下键/值对：

- `content`：视频的云存储路径。
- `mimeType`：内容类型。在我们的示例中，它是一个`avi`文件。
- `timeSegmentStart`：视频中要进行预测的开始时间戳。*注意*，时间戳必须以字符串形式指定，并跟随s（秒）、m（分钟）或h（小时）。
- `timeSegmentEnd`：视频中要进行预测的结束时间戳。

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {
        "content": test_item_1,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")
    data = {
        "content": test_item_2,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

进行批量预测请求

现在您的模型资源已经经过训练，您可以通过调用batch_predict()方法进行批量预测，具有以下参数：

- `job_display_name`：用于批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `sync`：如果设置为True，则调用在等待异步批处理作业完成时阻止。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="traffic_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

示例输出：

    INFO：google.cloud.aiplatform.jobs：正在创建BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖关系完成。
    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob已创建。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    INFO：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob（'projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296'）
    INFO：google.cloud.aiplatform.jobs：查看Batch Prediction Job：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296当前状态：
    JobState.JOB_STATE_RUNNING

等待批量预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    信息：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    信息：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob运行完成。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的Cloud Storage输出存储桶中。您可以调用iter_outputs()方法获取生成结果的每个Cloud Storage文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
- `id`：每个预测请求的内部分配的唯一标识符。
- `displayName`：预测标签的类名。
- `confidences`：每个类别标签的预测置信度，介于0和1之间。
- `timeSegmentStart`：视频中到视频序列开始的时间偏移量。
- `timeSegmentEnd`：视频中到视频序列结束的时间偏移量。
- `frames`：被跟踪对象的帧的位置。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

{'instance': {'content': 'gs://automl-video-demo-data/traffic_videos/highway_005.mp4', 'mimeType': 'video/avi', 'timeSegmentStart': '0.0s', 'timeSegmentEnd': '5.0s'}, 'prediction': [{'id': '3767468895993069568', 'displayName': '轿车', 'timeSegmentStart': '5秒', 'timeSegmentEnd': '5秒', 'confidence': 0.61968166, 'frames': [{'timeOffset': '5秒', 'xMin': 0.62725407, 'xMax': 0.8197686, 'yMin': 0.5445565, 'yMax': 0.6752002}]}, {'id': '3767468895993069568', 'displayName': '轿车', 'timeSegmentStart': '0.100秒', 'timeSegmentEnd': '0.600秒', 'confidence': 0.5820198, 'frames': [{'timeOffset': '0.100秒', 'xMin': 0.82725096, 'xMax': 0.99906015, 'yMin': 0.6930067, 'yMax': 0.87328714}, {'timeOffset': '0.200秒', 'xMin': 0.7275455, 'xMax': 0.96733177, 'yMin': 0.56486815, 'yMax': 0.7271576}, {'timeOffset': '0.300秒', 'xMin': 0.6559273, 'xMax': 0.922594, 'yMin': 0.67381793, 'yMax': 0.87236154}, {'timeOffset': '0.400秒', 'xMin': 0.52804255, 'xMax': 0.8188649, 'yMin': 0.7136257, 'yMax': 0.9117379}, {'timeOffset': '0.500秒', 'xMin': 0.38841337, 'xMax': 0.68499833, 'yMin': 0.7241354, 'yMax': 0.9293221}, {'timeOffset': '0.600秒', 'xMin': 0.22706872, 'xMax': 0.5414151, 'yMin': 0.71608, 'yMax': 0.93481314}]}, {'id': '3767468895993069568', 'displayName': '轿车', 'timeSegmentStart': '3秒', 'timeSegmentEnd': '4秒', 'confidence': 0.49189684, 'frames': [{'timeOffset': '3秒', 'xMin': 0.7998861, 'xMax': 0.9865881, 'yMin': 0.6979363, 'yMax': 0.8698516}, {'timeOffset': '3.100秒', 'xMin': 0.7835889, 'xMax': 0.9862553, 'yMin': 0.7052906, 'yMax': 0.8742318}, {'timeOffset': '3.200秒', 'xMin': 0.7530042, 'xMax': 0.9377618, 'yMin': 0.5567795, 'yMax': 0.6924586}, {'timeOffset': '3.300秒', 'xMin': 0.6876858, 'xMax': 0.8742672, 'yMin': 0.5218122, 'yMax': 0.6595952}, {'timeOffset': '3.400秒', 'xMin': 0.62348735, 'xMax': 0.8095002, 'yMin': 0.5097083, 'yMax': 0.6480995}, {'timeOffset': ...

清理工作

要清理本项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

# Delete the dataset using the Vertex dataset object

dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline trainig job

dag.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI